In [41]:
import os
import datetime
import json
import time
# web scrape
import requests
import bs4

In [64]:
class Finder:
    """
    """
    
    def __init__(self,name,item_names,price_range,cities,distance):
        self.name = name
        self.item_names = item_names
        self.price_range = price_range
        self.cities = cities
        self.distance = distance
        
        self.search_url = 'https://{CITY}.craigslist.org/search/sss?hasPic=1&sort=rel&min_price={MIN_PRICE}&max_price={MAX_PRICE}&query={ITEM}&search_distance={DISTANCE}'
        self.urls = {
            city: [
                self.search_url.format(
                    CITY = city,
                    MIN_PRICE = self.price_range[0],
                    MAX_PRICE = self.price_range[1],
                    ITEM = item_name.replace(' ','%20'),
                    DISTANCE = self.distance,
                ) for item_name in self.item_names
            ] for city in self.cities
        }
        
        self.available = []
    
    def run(self):
        links = {}
        for city,urls in self.urls.items():
            links[city] = []
            for url in urls:
                r = requests.get(url)
                soup = bs4.BeautifulSoup(r.text,'html.parser')
                for row in soup.find_all(name='li', attrs={'class':'result-row'}):
                    links[city].append(row.find(name='a',attrs={'class':'result-title hdrlnk'})['href'])
        self.available = links

In [65]:
hobie_params = {
    'name': 'hobie cat',
    'price_range': (2000,5000),
    'cities': ['washingtondc','richmond','norfolk','fredericksburg','boston',],
    'distance': '50',
    'item_names': ['hobie cat','hobie cat wave 16']
}
hobie = Finder(**hobie_params)
hobie.run()
hobie.available

{'washingtondc': [],
 'richmond': ['https://richmond.craigslist.org/boa/d/mattaponi-hobie-cat-16/7486557753.html'],
 'norfolk': ['https://richmond.craigslist.org/boa/d/mattaponi-hobie-cat-16/7486557753.html'],
 'fredericksburg': ['https://richmond.craigslist.org/boa/d/mattaponi-hobie-cat-16/7486557753.html'],
 'boston': ['https://worcester.craigslist.org/boa/d/bolton-hobie-cat-18/7474916279.html',
  'https://worcester.craigslist.org/boa/d/bolton-hobie-cat-18/7474937166.html',
  'https://providence.craigslist.org/boa/d/portsmouth-hobie-wave-with-trailer/7484852900.html',
  'https://southcoast.craigslist.org/boa/d/portsmouth-hobie-wave-catamaran-with/7484848833.html']}